#### litert_CV 虛擬環境
* model

In [2]:
try:
    from tflite_runtime.interpreter import Interpreter
except ImportError:
    from tensorflow.lite.python.interpreter import Interpreter
import cv2
import numpy as np

model_path = "models/mobilenet_v1_1.0_224_quantized_1_metadata_1.tflite"
label_path = "models/labels.txt"
label_names = []
with open(label_path, "r") as f:
    for line in f.readlines():
        label_names.append(line.strip())
interpreter = Interpreter(model_path)
print("成功載入模型...")
interpreter.allocate_tensors()
_, height, width, _ = interpreter.get_input_details()[0]["shape"]
print("影像尺寸: (", width, ",", height, ")")
image = cv2.imread("images/dog.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = cv2.resize(image_rgb, (width, height))
input_data = np.expand_dims(image_resized, axis=0)
interpreter.set_tensor(
    interpreter.get_input_details()[0]["index"], input_data)
interpreter.invoke()
output_details = interpreter.get_output_details()[0]
output = np.squeeze(interpreter.get_tensor(output_details["index"]))
label_id = np.argmax(output)
scale, zero_point = output_details["quantization"]
prob = scale * (output[label_id] - zero_point)
classification_label = label_names[label_id]
print("分類名稱 =", classification_label)
print("影像可能性 =", np.round(prob*100, 2), "%")

cv2.imshow("Object Detector", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


成功載入模型...
影像尺寸: ( 224 , 224 )
分類名稱 = beagle
影像可能性 = 70.7 %


In [1]:
try:
    from tflite_runtime.interpreter import Interpreter
except ImportError:
    from tensorflow.lite.python.interpreter import Interpreter
import cv2
import numpy as np

model_path = "models/lite-model_ssd_mobilenet_v1_1_metadata_2.tflite"
label_path = "models/labelmap.txt"
label_names = []
with open(label_path, "r") as f:
    for line in f.readlines():
        label_names.append(line.strip())
interpreter = Interpreter(model_path=model_path)
print("成功載入模型...")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
_, height, width, _ = input_details[0]["shape"]
print("圖片資訊: (", width, ",", height, ")")
img = cv2.imread("images/people.jpg")
imgHeight, imgWidth, _ = img.shape
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, (width, height))
input_data = np.expand_dims(img_resized, axis=0)
interpreter.set_tensor(input_details[0]["index"], input_data)
interpreter.invoke()
boxes = interpreter.get_tensor(output_details[0]["index"])[0]
classes = interpreter.get_tensor(output_details[1]["index"])[0]
scores = interpreter.get_tensor(output_details[2]["index"])[0]
for i in range(len(scores)):
    if ((scores[i] > 0.5) and (scores[i] <= 1.0)):
        startY = int(max(1, (boxes[i][0] * imgHeight)))
        startX = int(max(1, (boxes[i][1] * imgWidth)))
        endY = int(min(imgHeight, (boxes[i][2] * imgHeight)))
        endX = int(min(imgWidth, (boxes[i][3] * imgWidth)))
        cv2.rectangle(img, (startX, startY), (endX, endY),
                      (10, 255, 0), 2)
        object_name = label_names[int(classes[i])]
        prob = np.round(scores[i]*100, 2)
        label = object_name + ": " + str(prob) + "%"
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(img, label, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (10, 255, 0), 2)

cv2.imshow("Object Detector", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


成功載入模型...
圖片資訊: ( 300 , 300 )


In [3]:
import face_recognition
import cv2

img = cv2.imread("images/faces2.jpg")
faces = face_recognition.face_locations(img,
                                        number_of_times_to_upsample=1,
                                        model="hog")
print("臉數=", len(faces))
for face in faces:
    top, right, bottom, left = face
    cv2.rectangle(img, (left, top), (right, bottom),
                  (0, 0, 255), 3)

cv2.imshow("Faces", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


c:\Users\q5j6j\miniconda3\envs\litert_CV\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


臉數= 2


In [4]:
import face_recognition
import cv2

img = cv2.imread("images/mary.jpg")
known_encoding = face_recognition.face_encodings(img)[0]
new_img = cv2.imread("images/mary2.jpg")
new_encoding = face_recognition.face_encodings(new_img)[0]

result = face_recognition.compare_faces([known_encoding],
                                        new_encoding, tolerance=0.6)
print(result)

[True]


In [1]:
# 多人臉比對範例

import face_recognition
import cv2
import os
import numpy as np

# 已知人物資料夾路徑
known_faces_dir = "known_faces"
tolerance = 0.6
model = "hog"  # 可改為 "cnn"（需 GPU）

# 載入已知人臉資料
print("載入已知人臉資料中...")
known_encodings = []
known_names = []

for filename in os.listdir(known_faces_dir):
    if not (filename.endswith(".jpg") or filename.endswith(".png")):
        continue
    path = os.path.join(known_faces_dir, filename)
    name = os.path.splitext(filename)[0]  # 檔名即人物名稱

    image = face_recognition.load_image_file(path)
    encodings = face_recognition.face_encodings(image)
    if len(encodings) > 0:
        known_encodings.append(encodings[0])
        known_names.append(name)
        print(f"已載入：{name}")
    else:
        print(f"無法偵測臉部：{filename}")

print(f"共載入 {len(known_encodings)} 位人物。")

# 讀取未知人臉影像
unknown_image = face_recognition.load_image_file("images/obama2.jpg")
unknown_locations = face_recognition.face_locations(unknown_image, model=model)
unknown_encodings = face_recognition.face_encodings(unknown_image, unknown_locations)

# 轉成 OpenCV 格式以顯示
unknown_image_bgr = cv2.cvtColor(unknown_image, cv2.COLOR_RGB2BGR)

# 比對每一張臉
for (top, right, bottom, left), face_encoding in zip(unknown_locations, unknown_encodings):
    results = face_recognition.compare_faces(known_encodings, face_encoding, tolerance)
    distances = face_recognition.face_distance(known_encodings, face_encoding)
    best_match_index = np.argmin(distances)

    if results[best_match_index]:
        name = known_names[best_match_index]
        label = f"{name} ({distances[best_match_index]:.2f})"
        color = (0, 255, 0)
    else:
        name = "Unknown"
        label = f"{name}"
        color = (0, 0, 255)

    # 繪製方框與姓名
    cv2.rectangle(unknown_image_bgr, (left, top), (right, bottom), color, 2)
    cv2.rectangle(unknown_image_bgr, (left, bottom - 25), (right, bottom), color, cv2.FILLED)
    cv2.putText(unknown_image_bgr, label, (left + 6, bottom - 6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

# 顯示結果
cv2.imshow("Face Recognition Result", unknown_image_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()


c:\Users\q5j6j\miniconda3\envs\litert_CV\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


載入已知人臉資料中...


FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'known_faces'